### Step 1
- Install the `databricks-industrial-automation-suite` Python package.
- _See documentation_: [PyPi](https://pypi.org/project/databricks-industrial-automation-suite/#description)

In [0]:
%pip install databricks-industrial-automation-suite

In [0]:
%pip list | grep databricks

### Step 2
- Setup a basic OPC UA Client.
- We can:
  * Connect to a remote OPC UA server via IP address (e.g. 132.65.41.78).
  * Use a prebuilt simulation.

In [0]:
oc = OPCUAClient(server_url="opc.tcp:// 132.65.41.78:4840/freeopcua/server/")